## Part 1: Explore the Data

Import the data and use Pandas to learn more about the dataset.

In [1]:
import pandas as pd

df = pd.read_csv('Resources/client_dataset.csv')

df.head()

,first,last,job,phone,email,client_id,order_id,order_date,order_week,order_year,item_id,category,subcategory,unit_price,unit_cost,unit_weight,qty,line_number
0,Donald,Harding,Immunologist,793-904-7725x39308,harding.donald.7185@sullivan.com,58515,8953482,2023-04-28,17,2023,EUD29711-63-6U,decor,wall art,1096.80,762.71,7.50,105,1
1,Tiffany,Myers,Music therapist,201.442.4543x942,myers.t.6537@ferguson-johnson.net,37609,8069089,2023-05-19,20,2023,XDA18116-89-4A,consumables,pens,24.95,15.09,1.49,21,0
2,Shannon,Watson,Immunologist,687.737.9424x8503,swatson8146@payne.net,57113,1902144,2023-01-29,4,2023,ABE59463-05-7E,software,project management,13.52,7.86,1.68,39,6
3,Nathan,Baker,Accounting technician,827-788-8123x012,bakernathan@benson.com,46554,9031802,2023-04-25,17,2023,ZMM00836-65-0C,consumables,pens,36.42,24.85,1.23,29,3
4,Christina,Schwartz,Chiropractor,265-829-3643,christinaschwartz9252@mcconnell.com,92089,1322274,2023-05-28,21,2023,BZX55559-12-3X,consumables,misc,195.10,108.17,46.43,20,1


In [2]:
# View the column names in the data
df.columns

Index(['first', 'last', 'job', 'phone', 'email', 'client_id', 'order_id',
       'order_date', 'order_week', 'order_year', 'item_id', 'category',
       'subcategory', 'unit_price', 'unit_cost', 'unit_weight', 'qty',
       'line_number'],
      dtype='object')

In [3]:
# Use the describe function to gather some basic statistics
df.describe()

,client_id,order_id,order_week,order_year,unit_price,unit_cost,unit_weight,qty,line_number
count,54639.000000,5.463900e+04,54639.000000,54639.000000,54639.000000,54639.000000,54639.000000,5.463900e+04,54639.000000
mean,54837.869416,5.470190e+06,11.359139,2022.993064,136.267207,99.446073,5.004116,5.702646e+02,2.979667
std,25487.438231,2.599807e+06,7.023499,0.082997,183.873135,133.164267,5.326599,1.879552e+04,2.436320
min,10033.000000,1.000886e+06,1.000000,2022.000000,0.010000,0.010000,0.000000,0.000000e+00,0.000000
25%,33593.000000,3.196372e+06,6.000000,2023.000000,20.800000,14.840000,1.450000,3.200000e+01,1.000000
50%,53305.000000,5.496966e+06,11.000000,2023.000000,68.310000,49.890000,3.240000,6.800000e+01,3.000000
75%,78498.000000,7.733869e+06,17.000000,2023.000000,173.160000,125.570000,6.890000,1.700000e+02,5.000000
max,99984.000000,9.998480e+06,52.000000,2023.000000,1396.230000,846.270000,46.430000,3.958244e+06,9.000000


In [4]:
# Use this space to do any additional research
# and familiarize yourself with the data.
df.isnull().sum()

first          0
last           0
job            0
phone          0
email          0
client_id      0
order_id       0
order_date     0
order_week     0
order_year     0
item_id        0
category       0
subcategory    0
unit_price     0
unit_cost      0
unit_weight    0
qty            0
line_number    0
dtype: int64

In [5]:
# What three item categories had the most entries?
df['category'].value_counts().head(3)

category
consumables    23538
furniture      11915
software        8400
Name: count, dtype: int64

In [6]:
# For the category with the most entries,
# which subcategory had the most entries?
df['subcategory'].value_counts().head(1)

subcategory
bathroom supplies    6424
Name: count, dtype: int64

In [7]:
# Which five clients had the most entries in the data?
df['client_id'].value_counts().head(5)

client_id
33615    220
66037    211
46820    209
38378    207
24741    207
Name: count, dtype: int64

In [8]:
# Store the client ids of those top 5 clients in a list.
top_5 = [33615, 66037, 46820, 38378, 24741]
top_5

[33615, 66037, 46820, 38378, 24741]

In [9]:
# How many total units (the qty column) did the
# client with the most entries order order?
top_dog = df[df['client_id'] == 33615]
top_dog['qty'].sum()

64313

## Part 2: Transform the Data
Do we know that this client spent the more money than client 66037? If not, how would we find out? Transform the data using the steps below to prepare it for analysis.

In [10]:
# Create a column that calculates the 
# subtotal for each line using the unit_price
# and the qty
df['line_subtotal'] = df['unit_price'] * df['qty']

In [11]:
# Create a column for shipping price.
# Assume a shipping price of $7 per pound
# for orders over 50 pounds and $10 per
# pound for items 50 pounds or under.
def calculate_shipping(unit_weight, qty):
    total_weight = unit_weight * qty
    if total_weight > 50:
        return total_weight * 7
    else:
        return total_weight * 10

df['shipping_price'] = df.apply(lambda row: calculate_shipping(row['unit_weight'], row['qty']), axis=1)

In [12]:
# Create a column for the total price
# using the subtotal and the shipping price
# along with a sales tax of 9.25%
df['line_price'] = (df['line_subtotal'] + df['shipping_price']) * 1.0925

In [13]:
# Create a column for the cost
# of each line using unit cost, qty, and
# shipping price (assume the shipping cost
# is exactly what is charged to the client).
df['line_cost'] = (df['unit_cost'] * df['qty']) + df['shipping_price']

In [14]:
# Create a column for the profit of
# each line using line cost and line price
df['line_profit'] = df['line_price'] - df['line_cost']

In [15]:
df.head()

,first,last,job,phone,email,client_id,order_id,order_date,order_week,order_year,...,unit_price,unit_cost,unit_weight,qty,line_number,line_subtotal,shipping_price,line_price,line_cost,line_profit
0,Donald,Harding,Immunologist,793-904-7725x39308,harding.donald.7185@sullivan.com,58515,8953482,2023-04-28,17,2023,...,1096.80,762.71,7.50,105,1,115164.00,5512.50,131839.076250,85597.05,46242.026250
1,Tiffany,Myers,Music therapist,201.442.4543x942,myers.t.6537@ferguson-johnson.net,37609,8069089,2023-05-19,20,2023,...,24.95,15.09,1.49,21,0,523.95,312.90,914.258625,629.79,284.468625
2,Shannon,Watson,Immunologist,687.737.9424x8503,swatson8146@payne.net,57113,1902144,2023-01-29,4,2023,...,13.52,7.86,1.68,39,6,527.28,458.64,1077.117600,765.18,311.937600
3,Nathan,Baker,Accounting technician,827-788-8123x012,bakernathan@benson.com,46554,9031802,2023-04-25,17,2023,...,36.42,24.85,1.23,29,3,1056.18,356.70,1543.571400,1077.35,466.221400
4,Christina,Schwartz,Chiropractor,265-829-3643,christinaschwartz9252@mcconnell.com,92089,1322274,2023-05-28,21,2023,...,195.10,108.17,46.43,20,1,3902.00,6500.20,11364.403500,8663.60,2700.803500


## Part 3: Confirm your work
You have email receipts showing that the total prices for 3 orders. Confirm that your calculations match the receipts. Remember, each order has multiple lines.

Order ID 2742071 had a total price of \$152,811.89

Order ID 2173913 had a total price of \$162,388.71

Order ID 6128929 had a total price of \$923,441.25


In [16]:
filtered_rows = df[df['order_id'] == 2742071]
order_1 = filtered_rows['line_price'].sum()

filtered_rows_2 = df[df['order_id'] == 2173913]
order_2 = filtered_rows_2['line_price'].sum()

filtered_rows_3 = df[df['order_id'] == 6128929]
order_3 = filtered_rows_3['line_price'].sum()

In [17]:
# Check your work using the totals above
print(f'Order 2742071 total: ${order_1:.2f}')
print(f'Order 2173913 total: ${order_2:.2f}')
print(f'Order 6128929 total: ${order_3:.2f}')

Order 2742071 total: $152811.90
Order 2173913 total: $162388.72
Order 6128929 total: $923441.24


## Part 4: Summarize and Analyze
Use the new columns with confirmed values to find the following information.

In [18]:
client_qty_sum = df.groupby('client_id')['qty'].sum()
top_dogs = client_qty_sum.sort_values(ascending=False).head(5)
top_dogs

client_id
78965    4033302
86721    1010426
30143     846069
85931     644423
77354     506382
Name: qty, dtype: int64

In [19]:
client_1 = df[df['client_id'] == 78965]
client_11 = client_1['line_price'].sum()

client_2 = df[df['client_id'] == 86721]
client_22 = client_2['line_price'].sum()

client_3 = df[df['client_id'] == 30143]
client_33 = client_3['line_price'].sum()

client_4 = df[df['client_id'] == 85931]
client_44 = client_4['line_price'].sum()

client_5 = df[df['client_id'] == 77354]
client_55 = client_5['line_price'].sum()

In [20]:
# How much did each of the top 5 clients by quantity
# spend? Check your work from Part 1 for client ids.
print(f'78965: ${client_11:.2f}')
print(f'86721: ${client_22:.2f}')
print(f'30143: ${client_33:.2f}')
print(f'85931: ${client_44:.2f}')
print(f'77354: ${client_55:.2f}')

78965: $655923670.06
86721: $181266297.07
30143: $33328849.83
85931: $237417001.42
77354: $71942651.78


In [21]:
# Create a summary DataFrame showing the totals for the
# for the top 5 clients with the following information:
# total units purchased, total shipping price,
# total revenue, and total profit. Sort by total profit.
pd.set_option('display.float_format', lambda x: '%.2f' % x)
client_ids = [78965, 86721, 30143, 85931, 77354]
filtered_df = df[df['client_id'].isin(client_ids)]
grouped_df = filtered_df.groupby('client_id')[['qty', 'shipping_price', 'line_price', 'line_cost', 'line_profit']].sum()
grouped_df = grouped_df.sort_values(by='line_profit', ascending=False)
grouped_df = grouped_df.reset_index()

grouped_df

,client_id,qty,shipping_price,line_price,line_cost,line_profit
0,78965,4033302,5340756.88,655923670.06,408570827.59,247352842.47
1,85931,644423,10122549.19,237417001.42,128122872.60,109294128.82
2,86721,1010426,64335727.72,181266297.07,138291601.58,42974695.49
3,77354,506382,23814685.86,71942651.78,59510391.67,12432260.11
4,30143,846069,8981905.24,33328849.83,22078463.94,11250385.89


In [22]:
# Format the data and rename the columns
# to names suitable for presentation.
# Currency should be in millions of dollars.
new_df = grouped_df.rename(columns={
    'client_id': 'Client ID',
    'qty': 'Units',
    'shipping_price': 'Shipping',
    'line_price': 'Total Revenue',
    'line_cost': 'Total Cost',
    'line_profit': 'Total Profit'
})

for col in ['Shipping', 'Total Revenue', 'Total Cost', 'Total Profit']:
    new_df[col] = new_df[col].apply(lambda x: f'${x / 1e6:.2f}M')

new_df

,Client ID,Units,Shipping,Total Revenue,Total Cost,Total Profit
0,78965,4033302,$5.34M,$655.92M,$408.57M,$247.35M
1,85931,644423,$10.12M,$237.42M,$128.12M,$109.29M
2,86721,1010426,$64.34M,$181.27M,$138.29M,$42.97M
3,77354,506382,$23.81M,$71.94M,$59.51M,$12.43M
4,30143,846069,$8.98M,$33.33M,$22.08M,$11.25M


In [23]:
# Sort the updated data by "Total Profit" form highest to lowest
# If they weren't in order in the first place!
new_df['Total Profit'] = new_df['Total Profit'].replace('[\$,M]', '', regex=True).astype(float)
newer_df = new_df.sort_values('Total Profit', ascending=False)
newer_df['Total Profit'] = newer_df['Total Profit'].apply(lambda x: f'${x:.2f}M')
newer_df

,Client ID,Units,Shipping,Total Revenue,Total Cost,Total Profit
0,78965,4033302,$5.34M,$655.92M,$408.57M,$247.35M
1,85931,644423,$10.12M,$237.42M,$128.12M,$109.29M
2,86721,1010426,$64.34M,$181.27M,$138.29M,$42.97M
3,77354,506382,$23.81M,$71.94M,$59.51M,$12.43M
4,30143,846069,$8.98M,$33.33M,$22.08M,$11.25M


We can see that our client with the most orders is bringing us the most amount of money. The total profit made from sales to Cleint 78965 is far higher than the others. There is a huge profit drop off from our 1st, to 2nd, to 3rd most profitable client. It's also not necessarily the client who orders the most units that brings the most profits. For example the second and fourth client didn't have such a massive difference in units sold. The difference in profit is quite dramatic on the other hand.